# Basic regression with ARIMA errors.

This notebook contains an implemenation of regression with ARIMA errors.

>💡 It is recommended that you **study the Prophet exercises** in the next computer lab before going through this notebook.  The setup of this model is more complex and has been implemented to mimic Prophet.

🎓 **In this practical you will learn**

* ✅ How to setup Seasonal Indexes for Regression with ARIMA errors
* ✅ How to train a Regression with ARIMA errors model using `pmdarima` 
* ✅ Create a future data frame of Seasonal Indexes forecast ahead.
* 🎁 **Bonus**: Use a simple temporal train-test split design to calculate error metrics.

## Overview of notebook

In this implementation, months of the year and weeks of the year are represented by seasonal indexes (dummy variables either 0 or 1).

The key functions in the implementation are:


| Function Name         | Description                                                                                                                                                               |
|-----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| get_seasonal_indexes  | Takes a pandas.DatetimeIndex as input and returns a pandas.DataFrame containing dummy variable columns (0 or 1) representing the months of the year and days of the week. |
| make_future_dataframe | Generates the necessary future values for the seasonal indexes (dummy variables) required to make predictions for upcoming time periods.                                  |


## Imports

In [ ]:
import pandas as pd
import numpy as np

from forecast_tools.metrics import (
    mean_absolute_error, 
    absolute_coverage_difference
)
from forecast_tools.datasets import load_emergency_dept
from forecast_tools.plotting import plot_time_series

from pmdarima import auto_arima, ARIMA

from typing import Optional

import warnings
warnings.filterwarnings('ignore')

## Helper functions

In [ ]:
def preds_as_series(data, preds):
    '''
    Helper function for plotting predictions.
    Converts a numpy array of predictions to a 
    pandas.DataFrame with datetimeindex
    
    Parameters
    -----
    data - arraylike - the training data
    preds - numpy.array, vector of predictions 
    
    Returns:
    -------
    pandas.DataFrame
    '''
    start = pd.date_range(start=data.index.max(), periods=2, 
                          freq=data.index.freq).max()
    idx = pd.date_range(start=start, periods=len(preds), freq=data.index.freq)
    return pd.DataFrame(preds, index=idx)

In [ ]:
def intervals_as_dataframe(
    intervals: np.ndarray, 
    test: pd.DataFrame
) -> pd.DataFrame:
    '''
    Helper function for plotting predictions.
    
    Converts 2D numpy array containing prediction intervals to a 
    pandas.DataFrame with datetimeindex.

    Parameters:
    ----------
    intervals: np.ndarray 
        matrix of prediction intervals

    test: pd.DataFrame
        test in a dataframe format. This is passed to extract the datetimeindex
        to use in the new intervals dataframe.
    '''
    formatted_intervals = pd.DataFrame(
        intervals, 
        index=test.index, 
        columns=["lower", "upper"]
    )

    return formatted_intervals

In [ ]:
def plot_prediction_intervals(train, preds, intervals, 
                              test=None, show_train_size=None, figsize=(12,4)):
    '''
    Helper function to plot training data, point preds
    and 2 sets of prediction intevals
    
    assume 2 sets of PIs are provided!
    '''
    
    if show_train_size is None:
        show_train_size = len(train)
        
    train = train[-show_train_size:]
    ax = train.plot(figsize=figsize)
    

    mean = preds_as_series(train, preds)
    intervals_80 = preds_as_series(train, intervals[0])
    intervals_90 = preds_as_series(train, intervals[1])

    mean.plot(ax=ax, label='point forecast')

    ax.fill_between(intervals_80.index, mean[0], intervals_80[1], 
                    alpha=0.2,
                    label='80% PI', color='yellow');

    ax.fill_between(intervals_80.index,mean[0], intervals_80[0], 
                    alpha=0.2,
                    label='80% PI', color='yellow');

    ax.fill_between(intervals_80.index,intervals_80[1], intervals_90[1], 
                    alpha=0.2,
                    label='90% PI', color='purple');

    ax.fill_between(intervals_80.index,intervals_80[0], intervals_90[0], 
                    alpha=0.2,
                    label='90% PI', color='purple');
    
    if test is None:
        ax.legend(['train', 'point forecast', '80%PI', '_ignore','_ignore', 
                   '90%PI'], loc=2)
    else:
        test.plot(ax=ax, color='black', marker='o', ls='')
        ax.legend(['train', 'point forecast', 'Test', '80%PI', 
                   '_ignore','_ignore', '90%PI'], loc=2)

# Function to get seasonal indexes

This is the reusable function that provides the seasonal indexes for your Regression model.

🧗🏆💪 **Challenge:** An alternative to dummy indexes for months is to implement Fourier Series.

In [ ]:
def get_seasonal_indexes(
    idx: pd.DatetimeIndex, 
    include_month: Optional[bool] = True, 
    include_dow: Optional[bool] = True
) -> pd.DataFrame:
    """
    Given a DatetimeIndex, return the Seasonal Indexes (dummy variables) for use 
    with regression.
    
    Ensures a consistent feature set shape even if the input index does not 
    cover all months or days (e.g., short time series).
    
    Parameters
    ----------
    idx : pd.DatetimeIndex
        Dates included in the dataframe.
        
    include_month : bool, optional (default=True)
        If True, include 11 dummy variables for month of year (drop Jan).
        
    include_dow : bool, optional (default=True)
        If True, include 6 dummy variables for day of week (drop Mon).
        
    Returns
    -------
    pd.DataFrame
        DataFrame with index matching `idx` and integer dummy columns.
        
    Notes
    -----
    Modified for 2026:
    - Enforced categorical types to prevent missing columns on sparse data.
    - Optimized concatenation to run once.
    - Updated docstrings to NumPy style.
    """
    features = []

    # Month dummies: Always cover 1-12
    if include_month:
        month_cat = pd.Categorical(
            idx.month, 
            categories=range(1, 13), 
            ordered=True
        )
        features.append(
            pd.get_dummies(
                month_cat, 
                prefix='m', 
                drop_first=True, 
                dtype=int
            )
        )
        
    # Day of Week dummies: Always cover 0 (Mon) - 6 (Sun)
    if include_dow:
        dow_cat = pd.Categorical(
            idx.dayofweek, 
            categories=range(7), 
            ordered=True
        )
        features.append(
            pd.get_dummies(
                dow_cat, 
                prefix='dow', 
                drop_first=True, 
                dtype=int
            )
        )

    if not features:
        return pd.DataFrame(index=idx)

    # Concatenate all features at once and align index
    seasonal_idx = pd.concat(features, axis=1)
    seasonal_idx.index = idx
    
    return seasonal_idx

## Example with `forecast_tools` ED dataset

In this example we will forecast 28 days of daily ED attendances using `forecast-tools` built in ED dataset.

In [ ]:
TARGET = 0.80
HOLDOUT = 28
PERIOD = 7

# built in dataset
attends = load_emergency_dept()

# temporal train-test split
train = attends[:-HOLDOUT]
test = attends[-HOLDOUT:]

X = get_seasonal_indexes(train.index)

# quick look at the tail
X.tail(7)

## Use auto_arima with one small change

Pass in the regressors in X 

> Note that in earlier version of `pmdarima` the `X` regressors are known as exogenous variables.

To use the `auto_arima` function you would use the following code:

```python
model = auto_arima(train, X=X, m=PERIOD, d=1, supress_warnings=True, 
                   maxiter=100)
```

For me this returned an ARIMA(5,1,1) with an intercept. To save time the code below creates the model directly using `ARIMA`

In [ ]:
# uncomment this code if you want to run auto_arima
# it will take a few minutes to run and fit all models.
# model = auto_arima(train, X=X, m=PERIOD, d=1, supress_warnings=True, 
#                    maxiter=100)

# model

In [ ]:
# create ARIMA model
model = ARIMA(
    order=(5,1,1), 
    seasonal_order=(0, 0, 0, 7),
    with_intercept=True,
    suppress_warnings=True, 
    maxiter=100
)

# fit the data including seasonal indexes
model.fit(train, X=X)

model

In [ ]:
model.summary()

## Making a forecast

We will do this similar to Prophet and create a "future" dataframe that holds all of our regressors for the period we want to predict.

In [ ]:
def make_future_dataframe(h, y_train, include_mth=True, include_dow=True):
    '''
    Make a dataframe h steps into the future of y_train
    
    Params:
    ------
    h: int
        Forecast horizon
        
    y_train: pd.DataFrame
        Dataframe containing training data.  Must have a DataTimeIndex
    
    '''
    idx = pd.date_range(start=y_train.iloc[-1].name, periods=y_train.shape[0]+h, 
                        freq='D')
    seasonal_idxs = get_seasonal_indexes(idx, include_month=True, 
                                         include_dow=True)
    return seasonal_idxs.iloc[-h:]


In [ ]:
# future_dataframe just holds our X regression i.e. seasonal indexes in the future
future_dataframe = make_future_dataframe(HOLDOUT, train)
future_dataframe.head()

The forecast function is just for convenience.  It calls the model.predict method

In [ ]:
def forecast(model, future_dataframe, return_predict_int=True, alpha=0.05):
    '''
    Forecast with regression with ARIMA errors
    
    Params:
    ------
    h: int
        Forecast horizon
        
    future_dataframe: pd.DataFrame
        Future dataframe containing datetimeindex + seasonal indexes
    
    return_predict_int: bool, optional (default=True)
        Prediction interval with predictions
        
    alpha: float, optional (Deault=0.05)
        1 - coverage for prediction interval
        
    Returns:
    --------
    preds, intervals
    '''
    
    h = future_dataframe.shape[0]
    return model.predict(n_periods=h, X=future_dataframe, 
                         return_conf_int=return_predict_int, 
                         alpha=alpha)

In [ ]:
# Forecast and get 80% and 95% prediction intervals 
preds, intervals_95 = forecast(model, future_dataframe, alpha=0.05)
preds, intervals_80 = forecast(model, future_dataframe, alpha=0.2)
intervals = np.array([intervals_80, intervals_95])

## Visualise the forecast

Here we will use the `plot_time_series` function to view the point forecasts and 80 and 95% prediction intervals.

In [ ]:
# plot the prediction intervals

alphas = [0.2, 0.05]

# Create dictionary mapping coverage (80%', '95%') to DataFrames
intervals_dict = {
    f'{1-alpha:.0%} PI': intervals_as_dataframe(interval, test)
    for alpha, interval in zip(alphas, intervals)
}

# View prediction and intervals.
_ = plot_time_series(
    training_data = train[-60:],
    test_data = test,
    forecast = pd.DataFrame(preds, index=test.index),
    prediction_intervals = intervals_dict,
    y_axis_label="ED Attendances",

)

## Evaluate forecast

Once you have a prediction, we follow standard approaches to evaluate it. Here we have set the problem up using a simple temporal train-test split. 

> In practice, we should compare this to a Naive forecast and also a basic ARIMA model.

In [ ]:
# MAE
mae = mean_absolute_error(y_true=test, y_pred=preds)

# ACD for 95% intervals
acd_95 = absolute_coverage_difference(
    y_true=test, 
    intervals=intervals_dict["95% PI"]
)

print(f"MSE: {round(mae, 1)}")
print(f"ACD 95% PIs: {round(acd_95, 2)}")